In [1]:
# baseline cnn model for digit
import numpy as np
#import import_ipynb
from loader import Loader
from sklearn.model_selection import KFold
from keras.utils import to_categorical
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import BatchNormalization
import random as r
import matplotlib.pyplot as plt



In [2]:
def dataset1():
# Dividing the data into train and test data
    dataset = Loader()
    #trainX, trainY = dataset.getWholeTrainSet(pca=False)
    trainX, trainY = dataset.augment()
    # noisy dataset for training only
    #testX, testY = dataset.getNoisySet(intensity=0.5)
    testX, testY = dataset.getWholeTestSet(pca=False)
    trainX = np.array(trainX)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)
    

    testX = testX.reshape((testX.shape[0], 16,15, 1))
    trainX = trainX.reshape((trainX.shape[0], 16,15, 1))
    testY = to_categorical(testY)
    trainY = to_categorical(trainY)
    return trainX, trainY, testX, testY



In [3]:
def getdataset():
    
    trainX, trainY, testX, testY = dataset1()
    return trainX, trainY, testX, testY

In [4]:
#Data normalization
def normalise_data(traindata, testdata):
    traindata = traindata.astype('float32')
    testdata = testdata.astype('float32')
    train_norm = traindata / 6.0
    
    # check if test data is on a range of 0-255
    test_norm = testdata / 6.0
    return train_norm, test_norm

In [5]:
#trainX, trainY, testX, testY = getdataset()
#train_norm, test_norm  = normalise_data(trainX, testX)
#print(train_norm[59])


In [6]:
def model_definition():
    # tune model parameters here
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(16,15, 1)))
    #model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    
    #model.add(Conv2D(32, (3, 3),activation='relu', kernel_initializer='he_uniform'))
    #model.add(BatchNormalization())
    #model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    
    
    
    # Flatten and apply drop out or apply drop out after Conv2D it is essentially the same
    model.add(Dropout(0.50))
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
    
    model.add(Dropout(0.20))
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
    model.add(Dropout(0.20))
    model.add(Dense(10, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
    
    
    
#    model.add(Dropout(0.20))
    model.add(Dense(10, activation='softmax'))
    #opt = SGD(lr=0.01)
    opt = SGD(lr=0.03, momentum=0.9)
    #opt = 'adam'
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
def train_modelkfold(mtrainX, mtrainY, folds=10):
    accuracies,histories = list(), list()
    
    # random shuffle k fold validation
    kfold = KFold(folds, shuffle=True, random_state=3)
    for training_indices, testing_indices in kfold.split(mtrainX):
        model = model_definition()
        trainX, trainY, testX, testY = mtrainX[training_indices], mtrainY[training_indices], mtrainX[testing_indices], mtrainY[testing_indices]
        # so this model.fit reflects the model.fit for only the kfold 
        #model.fit(trainX, trainY, epochs=120, batch_size=10, validation_data=(testX, testY), verbose=0)
        history_kfold = model.fit(trainX, trainY, epochs=150, batch_size=10, validation_data=(testX, testY), verbose=0)
        
        # returning the validation accuracies of every k fold validation set
        _, accuraccy = model.evaluate(testX, testY, verbose=0)
        print('%.3f' % (accuraccy * 100.0))
        # stores accuracies and test data on last kth model
        accuracies.append(accuraccy)
        histories.append(history_kfold)
        # implement a list for storing all the k fold models. If you wish
    return accuracies,histories

In [8]:
# function to print the accuracies of the k fold iterations
def run_model():   
    # runnning k fold
    trainX, trainY, testX, testY = getdataset()
    trainX,testX = normalise_data(trainX,testX)
    # printing the accuracies of the k fold
    print('printing the k fold accuracies')
    accuracies, histories = train_modelkfold(trainX, trainY)
    
    
    
    print('Training on the whole dataset')
    history_trailist = list()
    # redefiniton of the model
    model = model_definition()
    history_training = model.fit(trainX, trainY, epochs=150, batch_size=32, verbose=0)
    
    history_trailist.append(history_training)
    # save model
    model.save('final_model.h5')
    #_, accuracy1 = model.evaluate(testX, testY, verbose=0)
    #print('Test accuracy =',accuracy1*100.0)
    
    
    # running on the saved model on the test set
    print('Testing the saved model')
    model = load_model('final_model.h5')
    # evaluate model on test dataset
    _, accuracy1 = model.evaluate(testX, testY, verbose=0)
    print('Test accuracy > %.3f' % (accuracy1 * 100.0))


In [9]:
run_model()

[39, 19, 89, 93, 37, 40, 61, 82, 83, 84, 68, 36, 8, 43, 13, 12, 9, 71, 15, 76, 54, 92, 32, 44, 16, 55, 74, 81, 25, 20]
(1000, 16, 15)
printing the k fold accuracies
98.000
98.000
95.000
98.000
98.000
97.000
96.000
97.000
96.000
99.000
Training on the whole dataset
Testing the saved model
Test accuracy > 98.400
